# LLM Proxy User Documentation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/derickson/ExpressOpenAIChatProxy/blob/main/Notebooks/LLM_Proxy.ipynb)

In [ ]:
! pip install -q openai
! pip install -q langchain

## How to use the proxy with OpenAI

In [ ]:
from getpass import getpass

my_proxy_key = getpass("Enter the instructor provided proxy key: ")
my_short_id = input("Enter a short user id to uniquely identify you: ")
my_combo_key = f"{my_proxy_key}-{my_short_id}"

In [ ]:
import os
import openai

# Load your API key from an environment variable or secret management service
openai.api_key = my_combo_key
openai.api_base = "https://llmproxy.gcp.elasticsa.co/v1"

chat_completion = None
try:
  chat_completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "user", "content": "Hello world"}
          ]
      )
  print(chat_completion)
except openai.error.OpenAIError as e:
    # Handle exceptions here, if any
    print("Error occurred:", e)

## How to use the proxy with LangChain

In [ ]:
import os
import openai
from langchain.schema import HumanMessage
from langchain.chat_models import ChatOpenAI
openai.api_key = my_combo_key
openai.api_base = "https://llmproxy.gcp.elasticsa.co/v1"

os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ["OPENAI_API_BASE"] = openai.api_base

openai_chat = ChatOpenAI()

PROMPT = "Hello!"

response = openai_chat([HumanMessage(content=PROMPT)]).content
print(response)

### Big Chat Loop with LangChain

this is the last step of the lab part 1. Let's chat with an AI bot.

In [ ]:
from langchain import ConversationChain
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory

import json
# pretty printing JSON objects
def json_pretty(input_object):
  print(json.dumps(input_object, indent=4))


import textwrap
# wrap text when printing, because colab scrolls output to the right too much
def wrap_text(text, width):
    wrapped_text = textwrap.wrap(text, width)
    return '\n'.join(wrapped_text)

template = """The following is a serious conversation between a human and a TV
News Anchor named Newsy McNewserson.
The Anchor provides autoritative information and commentary in short responses.
If the Anchor does not know the answer to a question,
he truthfully says it does not know.

Current conversation:
{history}
Human: {input}
Anchor:"""

MEMORY = ConversationBufferWindowMemory(ai_prefix="Anchor", k=2)

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=openai_chat,
    verbose=False,
    memory=MEMORY,
)

def chatLoop():
  print(" -- Have a conversation with a TV news Anchor: ")
  print(" -- Ask this AI \"what is in the news?\" ")
  print(" -- type 'exit' when done")

  user_input = input("> ")
  while not user_input.lower().startswith("exit"):
      print( wrap_text(conversation.run(user_input),70) )
      print(" -- type 'exit' when done")
      user_input = input("> ")
  print("\n -- end conversation --")

In [ ]:
## start a new chat each time
MEMORY.clear()
## start the chat
chatLoop()